In [1]:
# https://habr.com/ru/post/347564/.
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"
os.environ['THEANO_FLAGS'] = "mode=FAST_RUN,device=cuda,floatX=float32"
import theano
print(theano.config.device, theano.config.floatX)

import keras
import keras_metrics
import sys
#from google.colab import files # для импорта данных в google colab
#from google.colab import drive # для импорта данных в google colab из google drive
import zipfile # для работы с архивами 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sn
import pickle
import csv
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from PIL import Image
import os
import tensorflow as tf

from keras.layers import Input
from keras.applications import xception
from keras.preprocessing import image

from tensorflow.keras.preprocessing import image as im
import time

from keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical
from tensorflow.keras.models import load_model  



Can not use cuDNN on context None: cannot compile with cuDNN. We got this error:
b'In file included from /tmp/try_flags_f20ew7by.c:4:0:\n/usr/include/cudnn.h:63:10: fatal error: driver_types.h: No such file or directory\n #include "driver_types.h"\n          ^~~~~~~~~~~~~~~~\ncompilation terminated.\n'
Mapped name None to device cuda: Tesla V100-SXM2-32GB (0000:04:00.0)


cuda float32


Using TensorFlow backend.


In [2]:
class_names = ['0', '1', '2', '3', '4', '5']
print(len(class_names))

6


In [3]:
K.set_learning_phase(1)

In [4]:
img_width, img_height = 50, 50
nb_train_samples = 26640
nb_validation_samples = 12630
epochs = 12
batch_size = 8
n_classes = 6

In [5]:
train_data_dir = "/datasets/DTLD/DTLD_crop/new_train/colours"    # дальше там папки, а не сами фотографии!!!!
validation_data_dir = "/datasets/DTLD/DTLD_crop/new_test/colours"

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    #shear_range=0.2,
    zoom_range=0.2,
    #fill_mode = 'constant',
    #cval = 1,
    rotation_range = 5,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip = False,
    vertical_flip = False)

test_datagen = ImageDataGenerator(rescale = 1.0 / 255)

In [6]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical')


Found 38301 images belonging to 6 classes.
Found 9111 images belonging to 6 classes.


In [7]:
def build_model_Xception():
  base_model = xception.Xception(include_top=True, 
                                      weights = None,
                                      input_tensor = None,
                                      input_shape=(img_width, img_height, 3),
                                      pooling=None,
                                      classes = 6)
  
  for layer in base_model.layers:
    layer.trainable = True
      
  model = Model(inputs = base_model.input, outputs = base_model.output)
  return model

In [8]:
model = build_model_Xception()
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc', 'mse'])

ValueError: Input size must be at least 71x71; got `input_shape=(50, 50, 3)`

In [17]:
early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, min_delta=1e-4) # остановка обучения, если loss на валидационном множесте улучшается менее чем на 10^-4
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_delta=1e-4)
filepath="weights-improvement-71-{epoch:02d}-{val_acc:.2f}.hdf5"
check = ModelCheckpoint(filepath, monitor = "val_acc", save_best_only = False) # сохранение лучшей (с наибольшим acc на валидационном множестве) сети
callbacks_list = [early_stop, reduce_lr , check]

In [ ]:
model_history = model.fit_generator(
    train_generator,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size,
    callbacks = callbacks_list,
    steps_per_epoch = nb_train_samples // batch_size )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/12
1898/3330 [================>.............] - ETA: 1:10:36 - loss: 0.5773 - acc: 0.8264 - mean_squared_error: 0.0422

In [12]:
callbacks_list

In [106]:
model = load_model('weights-improvement-12-0.97.hdf5')

In [107]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc', 'mse', precision, recall])

In [108]:
evaluation = model.evaluate_generator(validation_generator, verbose=1)

1139/1139 [==============================] - 1381s 1s/step - loss: 0.1343 - acc: 0.9663 - mean_squared_error: 0.0088 - precision: 0.9691 - recall: 0.9640


In [109]:
model.metrics_names

['loss', 'acc', 'mean_squared_error', 'precision', 'recall']

In [110]:
evaluation

[0.1342851871318884, 0.9663045, 0.00879281, 0.9690518, 0.9639879]

In [111]:
def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [112]:
def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall